### Importing Required Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Loading the active.csv file

In [2]:
df = pd.read_csv('active.csv')
df.head()
# status_changed_by__date format in YYYY-MM-DD

,user_id,status_changed_by__date,is_active
0,3176,2020-03-08,0
1,2830,2020-03-08,0
2,3614,2020-03-08,1
3,3614,2020-03-08,1
4,3614,2020-03-08,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26957 entries, 0 to 26956
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   user_id                  26957 non-null  int64 
 1   status_changed_by__date  26957 non-null  object
 2   is_active                26957 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 631.9+ KB


In [4]:
df.shape

(26957, 3)

### Droping duplicate value

##### I am droping the duplicate rows becasue they are giving the same information
##### and if I drop that then it will increase the computation time without any information loss.

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.shape

(3588, 3)

In [7]:
# 26957-3588=23,369, This is total no. of values that I droped. While keeping the one set of all duplicates.

In [8]:
#df.head(15)

In [9]:
# I sorted the rows by user_id and status_changed_by__date
# I did it becasuse there are some values of two column named user_id and is_active, which 
# capture same information multiple time
# Eg from the active.csv file
# user_id status_changed_by__date is_active

# 1       3/21/2020                0
# 1       3/25/2020                0  
# 1       3/28/2020                1

# 1       3/29/2020                0
# 1       4/13/2020                0
# 1       4/17/2020                0

# in the above data we can see that it capture the same information and the 2nd row is redundant. 
# So that we need to drop that value.
# and before droping that we need to do sorting 
# so that we can compare one row with previous row based on two column condition 
# (col1: user_id, col2:status_changed_by__date)
 

In [10]:
df.sort_values(by = ['user_id', 'status_changed_by__date'],inplace=True)
df.head()

,user_id,status_changed_by__date,is_active
939,1,2020-03-21,0
1244,1,2020-03-25,0
1506,1,2020-03-28,1
1521,1,2020-03-29,0
2490,1,2020-04-13,0


In [11]:
# here I are comparing user_id with previous user_id and same for is_active column and 
# when their difference is zero we are assigning 'UP' otherwise 'DOWN'
# I am creating new column Position beacuse it will help me in locating the rows where values are not changing

In [12]:
df['Position'] = np.where((df['user_id'].diff().eq(0)) & (df['is_active'].diff().eq(0)),"UP","DOWN")
df.head()

,user_id,status_changed_by__date,is_active,Position
939,1,2020-03-21,0,DOWN
1244,1,2020-03-25,0,UP
1506,1,2020-03-28,1,DOWN
1521,1,2020-03-29,0,DOWN
2490,1,2020-04-13,0,UP


In [13]:
# Now i am fillitering out all the rows where positin = 'UP'

In [14]:
df = df[df['Position'] != 'UP']

In [15]:
df.head()  # Now we can see that Position column has down values only

,user_id,status_changed_by__date,is_active,Position
939,1,2020-03-21,0,DOWN
1506,1,2020-03-28,1,DOWN
1521,1,2020-03-29,0,DOWN
3144,1,2020-04-24,1,DOWN
4549,1,2020-05-01,0,DOWN


In [16]:
df = df.drop('Position', 1)  # There is no need for position column anymore. we used it for reference purpose only
                             # so, That's why I dropped it
df.head()

,user_id,status_changed_by__date,is_active
939,1,2020-03-21,0
1506,1,2020-03-28,1
1521,1,2020-03-29,0
3144,1,2020-04-24,1
4549,1,2020-05-01,0


In [17]:
# Now, I am going to extract the unique user_id
## I am doing this because for different user_id the status_changed_by__date are overlaping 
## and that is causing issue while expanding the status_changed_by__date column.

In [18]:
f = df.groupby(['user_id']) # It will help me using get method through which I can call each user_id iteratively

In [19]:
new_userid = df["user_id"]

In [20]:
new_userid 

939          1
1506         1
1521         1
3144         1
4549         1
         ...  
22645     8831
19328     8842
22778     8842
24602    11644
25069    11644
Name: user_id, Length: 3313, dtype: int64

In [21]:
unique_userid = list(set(new_userid)) #removing the duplicate value by using set then again converting to list data_Type

In [22]:
#print(unique_userid)

In [23]:
unique_userid.sort() # sorting the unique user_id by ascending order

In [24]:
#unique_userid

In [25]:
print(f.get_group(unique_userid[0])) # value coressponding to the user_id number =1

       user_id status_changed_by__date  is_active
939          1              2020-03-21          0
1506         1              2020-03-28          1
1521         1              2020-03-29          0
3144         1              2020-04-24          1
4549         1              2020-05-01          0
4687         1              2020-05-03          1
4876         1              2020-05-04          0
7684         1              2020-05-25          1
10971        1              2020-06-04          0
12746        1              2020-06-11          1
16670        1              2020-06-12          0
18948        1              2020-06-24          1
19872        1              2020-06-28          0
19885        1              2020-06-28          1
24923        1              2020-08-03          0
24929        1              2020-08-03          1
25407        1              2020-08-24          0
25410        1              2020-08-24          1
25621        1              2020-08-28          0


In [26]:
print(f.get_group(unique_userid[1])) # value coressponding to the user_id number =2

      user_id status_changed_by__date  is_active
3975       19              2020-04-26          0


In [27]:
for i in range(len(unique_userid)):   # callling all user_id data through for loop
    print(f.get_group(unique_userid[i]))
    print("=============================================================") # using this just for reference
                                                                            # after the seprator different user_id comes 
                                                                                    

       user_id status_changed_by__date  is_active
939          1              2020-03-21          0
1506         1              2020-03-28          1
1521         1              2020-03-29          0
3144         1              2020-04-24          1
4549         1              2020-05-01          0
4687         1              2020-05-03          1
4876         1              2020-05-04          0
7684         1              2020-05-25          1
10971        1              2020-06-04          0
12746        1              2020-06-11          1
16670        1              2020-06-12          0
18948        1              2020-06-24          1
19872        1              2020-06-28          0
19885        1              2020-06-28          1
24923        1              2020-08-03          0
24929        1              2020-08-03          1
25407        1              2020-08-24          0
25410        1              2020-08-24          1
25621        1              2020-08-28          0


       user_id status_changed_by__date  is_active
10141      710              2020-06-02          0
      user_id status_changed_by__date  is_active
8655      713              2020-05-28          0
      user_id status_changed_by__date  is_active
4836      737              2020-05-04          0
      user_id status_changed_by__date  is_active
167       769              2020-03-09          0
896       769              2020-03-20          1
963       769              2020-03-21          0
1239      769              2020-03-24          1
1426      769              2020-03-27          0
1734      769              2020-04-01          1
2175      769              2020-04-06          0
2528      769              2020-04-14          1
2622      769              2020-04-16          0
2786      769              2020-04-19          1
4535      769              2020-04-30          0
5321      769              2020-05-06          1
7743      769              2020-05-25          0
9675      769     

26952     2161              2020-09-29          0
       user_id status_changed_by__date  is_active
170       2174              2020-03-09          1
193       2174              2020-03-10          0
2685      2174              2020-04-16          1
2714      2174              2020-04-17          0
3374      2174              2020-04-25          1
4523      2174              2020-04-30          0
6739      2174              2020-05-18          1
7424      2174              2020-05-22          0
9602      2174              2020-05-30          1
10236     2174              2020-06-02          0
10490     2174              2020-06-03          1
       user_id status_changed_by__date  is_active
217       2208              2020-03-10          1
328       2208              2020-03-12          0
923       2208              2020-03-20          1
1203      2208              2020-03-24          0
1548      2208              2020-03-29          1
16991     2208              2020-06-13          0


20989     2622              2020-07-04          1
       user_id status_changed_by__date  is_active
12187     2656              2020-06-10          0
      user_id status_changed_by__date  is_active
214      2673              2020-03-10          0
268      2673              2020-03-11          1
434      2673              2020-03-14          0
672      2673              2020-03-17          1
805      2673              2020-03-19          0
1051     2673              2020-03-22          1
1350     2673              2020-03-26          0
1847     2673              2020-04-02          1
2128     2673              2020-04-05          0
3022     2673              2020-04-22          1
3845     2673              2020-04-26          0
4522     2673              2020-04-29          1
4537     2673              2020-04-30          0
5208     2673              2020-05-05          1
       user_id status_changed_by__date  is_active
1164      2691              2020-03-24          1
1263      2691 

26782     3521              2020-09-18          1
      user_id status_changed_by__date  is_active
2176     3530              2020-04-06          1
2299     3530              2020-04-08          0
       user_id status_changed_by__date  is_active
836       3538              2020-03-19          1
971       3538              2020-03-21          0
1315      3538              2020-03-25          1
1489      3538              2020-03-28          0
1672      3538              2020-03-31          1
1820      3538              2020-04-01          0
2242      3538              2020-04-07          1
2368      3538              2020-04-09          0
2919      3538              2020-04-21          1
17550     3538              2020-06-17          0
24442     3538              2020-07-21          1
24737     3538              2020-07-28          0
24888     3538              2020-08-01          1
       user_id status_changed_by__date  is_active
3130      3557              2020-04-23          1
455

...

17746     6364              2020-06-18          0
      user_id status_changed_by__date  is_active
6122     6365              2020-05-13          1
       user_id status_changed_by__date  is_active
11690     6366              2020-06-08          1
11847     6366              2020-06-09          0
17259     6366              2020-06-15          1
20398     6366              2020-07-01          0
22477     6366              2020-07-10          1
22910     6366              2020-07-17          0
       user_id status_changed_by__date  is_active
11011     6367              2020-06-04          1
11216     6367              2020-06-05          0
       user_id status_changed_by__date  is_active
21901     6370              2020-07-07          1
22129     6370              2020-07-09          0
       user_id status_changed_by__date  is_active
8318      6373              2020-05-27          1
9900      6373              2020-05-31          0
18230     6373              2020-06-19          1
18

8826     6572              2020-05-29          0
      user_id status_changed_by__date  is_active
6848     6577              2020-05-19          1
9709     6577              2020-05-30          0
       user_id status_changed_by__date  is_active
21433     6591              2020-07-05          1
21766     6591              2020-07-06          0
       user_id status_changed_by__date  is_active
7415      6598              2020-05-22          1
7748      6598              2020-05-25          0
10298     6598              2020-06-03          1
10977     6598              2020-06-04          0
20235     6598              2020-06-30          1
21077     6598              2020-07-04          0
       user_id status_changed_by__date  is_active
17411     6599              2020-06-16          1
17492     6599              2020-06-17          0
       user_id status_changed_by__date  is_active
18583     6611              2020-06-22          1
18989     6611              2020-06-24          0
    

26260     7348              2020-09-11          0
       user_id status_changed_by__date  is_active
12143     7352              2020-06-10          1
15217     7352              2020-06-11          0
25525     7352              2020-08-26          1
25655     7352              2020-08-28          0
       user_id status_changed_by__date  is_active
12092     7359              2020-06-10          1
15192     7359              2020-06-11          0
25924     7359              2020-09-10          1
26140     7359              2020-09-11          0
       user_id status_changed_by__date  is_active
11863     7363              2020-06-10          1
15194     7363              2020-06-11          0
25859     7363              2020-09-10          1
26124     7363              2020-09-11          0
       user_id status_changed_by__date  is_active
11907     7368              2020-06-10          1
15243     7368              2020-06-11          0
25863     7368              2020-09-10          1


13937     7624              2020-06-11          0
       user_id status_changed_by__date  is_active
11996     7625              2020-06-10          1
13949     7625              2020-06-11          0
25878     7625              2020-09-10          1
26370     7625              2020-09-11          0
       user_id status_changed_by__date  is_active
12029     7655              2020-06-10          1
14193     7655              2020-06-11          0
26894     7655              2020-09-25          1
       user_id status_changed_by__date  is_active
11969     7663              2020-06-10          1
14109     7663              2020-06-11          0
       user_id status_changed_by__date  is_active
11953     7669              2020-06-10          1
14084     7669              2020-06-11          0
25909     7669              2020-09-10          1
       user_id status_changed_by__date  is_active
12039     7670              2020-06-10          1
13606     7670              2020-06-11          0


14938     8036              2020-06-11          0
       user_id status_changed_by__date  is_active
12006     8090              2020-06-10          1
14888     8090              2020-06-11          0
25902     8090              2020-09-10          1
26615     8090              2020-09-11          0
       user_id status_changed_by__date  is_active
12093     8096              2020-06-10          1
14913     8096              2020-06-11          0
25880     8096              2020-09-10          1
26635     8096              2020-09-11          0
       user_id status_changed_by__date  is_active
12056     8134              2020-06-10          1
15039     8134              2020-06-11          0
       user_id status_changed_by__date  is_active
17538     8140              2020-06-17          1
17745     8140              2020-06-18          0
20155     8140              2020-06-29          1
20706     8140              2020-07-02          0
       user_id status_changed_by__date  is_active


In [28]:
df_final = pd.DataFrame()    # creating an empty dataframe for storing final result

for i in range(len(unique_userid)):
    dfi = f.get_group(unique_userid[i])
    
    dfi = dfi.set_index("status_changed_by__date") # changing index to date column
    
    dfi = dfi[~dfi.index.duplicated(keep='last')]  # droping when user_id and date is same    
    
    dfi.index = pd.to_datetime(dfi.index)  # conveting date whoes data_Type is STR to date_time data_Type                               
                                          # it will help in understanding date as date rather than some string values
        
    idx = pd.date_range(dfi.index[0],  dfi.index[-1]) # creating min and max date sequence for particular user_id
    
    dfi = dfi.reindex(idx, fill_value= None) # add the missing date along with
                                            # filling the extra values that created because of date expansion with nan
                                               
    df_final = df_final.append(dfi)  # appending the data to the final_dataframe

In [29]:
df_final.head(20)   # this is final_data frame, but we need to remove the index from date column

,user_id,is_active
2020-03-21,1.0,0.0
2020-03-22,NaN,NaN
2020-03-23,NaN,NaN
2020-03-24,NaN,NaN
2020-03-25,NaN,NaN
2020-03-26,NaN,NaN
2020-03-27,NaN,NaN
2020-03-28,1.0,1.0
2020-03-29,1.0,0.0
2020-03-30,NaN,NaN


In [30]:
# filling the nan values with forward values becasue they are corresponding to the same events.

In [31]:
df_final.fillna(method='ffill',inplace=True)

In [32]:
df_final.reset_index(inplace = True) # earlier the index was set to date column, that's why I am resting it.

In [33]:
df_final.rename(columns = {'index':'status_changed_by__date'}, inplace = True) # fixing date column name to original name
df_final = df_final[["user_id", "status_changed_by__date", "is_active"]]   # rearranging the column order

In [34]:
df_final.head(20)

,user_id,status_changed_by__date,is_active
0,1.0,2020-03-21,0.0
1,1.0,2020-03-22,0.0
2,1.0,2020-03-23,0.0
3,1.0,2020-03-24,0.0
4,1.0,2020-03-25,0.0
5,1.0,2020-03-26,0.0
6,1.0,2020-03-27,0.0
7,1.0,2020-03-28,1.0
8,1.0,2020-03-29,0.0
9,1.0,2020-03-30,0.0


In [35]:
# remove the hastag from next cell the get the required output file.

In [36]:
#df_final.to_csv("final_output_IN2.csv",index=False)

### follow up

In [37]:
# we can use Horizontal Partitioning on SQL Server tables by today date.
# SQL Server 2008 introduced a table partitioning wizard in SQL Server Management Studio.
# this provide UI for partiioning

In [38]:
#  PARTITION BY HASH(TO_DAYS(status_changed_by__date))  -- SQL code